## The _Kata_ of the Golden Rose

This _kata_ is a classic _refactoring_ exercise: we have a program that works, but it is necessary to add a new use case. For reasons that are self-explanatory by reading the program, part of it is required to be rewritten.

Originally conceived by [@TerryHughes](https://twitter.com/TerryHughes) and [@NotMyself](https://twitter.com/NotMyself) as an exercise in [_refactoring_ and C#](https://github.com/NotMyself/GildedRoseCore), [was extended to different languages](https://github.com/emilybache/GildedRose-Refactoring-Kata.git) and included in the book [Coding Dojo](https://leanpub.com/codingdojohandbook) by Emily Bache.

In this notebook you will find
- The original description of the code specifications, requirements and the new use case.
- The code itself to explore.

the mission is
- Write tests to prove that the code is actually working for the use cases for which it was designed.
- Write new tests to address the new case ('conjured' articles).
- Do the refactoring.

for this, [this repository](https://github.com/fcolavecchia/GildedRose-Kata-initial-fsharp.git) is provided, which contains three projects
- `GildedRose` with the original program.
- `GildedRose.UnitTests` where you can write the unit tests.
- `GildedRose.ApprovalTests` which contains the code approval test (for the original use cases.)

Fork accordingly and happy refactoring!





### Specifications of the Golden Rose (Gilded Rose)

Welcome to the **Gilded Rose** team.
As you may know, we are a small inn strategically located in a prestigious city, run by the friendly **Allison**.
We also buy and sell high quality merchandise.
Unfortunately, our merchandise is declining in quality as the sale date approaches.

We have a system installed that automatically updates the `inventory`.
This system was developed by a boy with little common sense named Leeroy, who is now off on new adventures.
Your task is to add a new feature to the system so that we can start selling a new category of items.

### Preliminary Description

But first, let's introduce the system:

* All items (`Item`) have a `sellIn` property that denotes the number of days we have to sell it
* All items have a `quality` property that denotes how valuable the item is
* At the end of each day, our system decrements both values ​​for each item using the `updateQuality` method

Pretty simple, right? Well, now is where it gets interesting:

* Once the recommended sell-by date has passed, `quality` degrades at twice the rate
* The `quality` of an article is never negative
* "Aged Brie Cheese" (`Aged brie`) increases its `quality` as it gets older
* Its `quality` increases by `1` unit every day
* after the `sale date` its `quality` increases by `2` units per day
* The `quality` of an article is never greater than `50`
* The item "Sulfuras" (`Sulfuras`), being a legendary item, does not change its `sales date` or degrade in `quality`
* A "Backstage Ticket", like brie, increases in `quality` as the `sell by` date approaches
* if the concert is 10 days or less away, the `quality` is increased by `2` units
* if there are 5 days or less left, the `quality` is increased by `3` units
* after the `sale date` the `quality` drops to `0`

### The requirement

We recently hired a vendor of *magically conjured* items.
This requires a system update:

* `Sorcered` items degrade their `quality` at twice the rate of normal items

Feel free to make any changes to the `updateQuality` message and add code as necessary, as long as everything is working correctly. However, **do not alter the `Item` object or its properties** as they belong to the goblin in that corner, who in a fit of rage will smack you down because he doesn't believe in codeshare culture .

### Endnotes

To clarify: an item can never have a `quality` higher than `50`, however Sulfuras being a legendary item has an immutable quality of `80`.

In [1]:
open System.Collections.Generic

type Item = 
    {
        Name: string
        SellIn: int 
        Quality: int 
    }

In [2]:
let Items = new List<Item>()
Items.Add({Name = "+5 Dexterity Vest"; SellIn = 10; Quality = 20})
Items.Add({Name = "Aged Brie"; SellIn = 2; Quality = 0})
Items.Add({Name = "Elixir of the Mongoose"; SellIn = 5; Quality = 7})
Items.Add({Name = "Sulfuras, Hand of Ragnaros"; SellIn = 0; Quality = 80})
Items.Add({Name = "Sulfuras, Hand of Ragnaros"; SellIn = -1; Quality = 80})
Items.Add({Name = "Backstage passes to a TAFKAL80ETC concert"; SellIn = 15; Quality = 20})
Items.Add({Name = "Backstage passes to a TAFKAL80ETC concert"; SellIn = 10; Quality = 49})
Items.Add({Name = "Backstage passes to a TAFKAL80ETC concert"; SellIn = 5; Quality = 49})
Items.Add({Name = "Conjured Mana Cake"; SellIn = 3; Quality = 6})

In [3]:
type GildedRose(items:IList<Item>) =
    let Items = items

    member this.UpdateQuality() =
        for i = 0 to Items.Count - 1 do
            if Items.[i].Name <> "Aged Brie" && Items.[i].Name <> "Backstage passes to a TAFKAL80ETC concert" then
                if Items.[i].Quality > 0 then
                    if Items.[i].Name <> "Sulfuras, Hand of Ragnaros" then
                        Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality - 1) } 
            else
               if Items.[i].Quality < 50 then
                    Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality + 1) } 
                    if Items.[i].Name = "Backstage passes to a TAFKAL80ETC concert" then
                        if Items.[i].SellIn < 11 then
                            if Items.[i].Quality < 50 then
                                Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality + 1) } 
                        if Items.[i].SellIn < 6 then
                            if Items.[i].Quality < 50 then
                                Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality + 1) } 
            if Items.[i].Name <> "Sulfuras, Hand of Ragnaros" then                 
                Items.[i] <- { Items.[i] with SellIn  = (Items.[i].SellIn - 1) } 
            if Items.[i].SellIn < 0 then
                if Items.[i].Name <> "Aged Brie" then
                    if Items.[i].Name <> "Backstage passes to a TAFKAL80ETC concert" then
                        if Items.[i].Quality > 0 then
                            if Items.[i].Name <> "Sulfuras, Hand of Ragnaros" then
                                Items.[i] <- { Items.[i] with Quality   = (Items.[i].Quality  - 1) } 
                    else
                        Items.[i] <- { Items.[i] with Quality   = (Items.[i].Quality  - Items.[i].Quality) } 
                else
                    if Items.[i].Quality < 50 then
                        Items.[i] <- { Items.[i] with Quality   = (Items.[i].Quality + 1) }  
        ()


In [4]:
let app = new GildedRose(Items)
for i = 0 to 30 do
    printfn "-------- day %d --------" i
    printfn "name, sellIn, quality"
    for j = 0 to Items.Count - 1 do
            printfn "%s, %d, %d" Items.[j].Name Items.[j].SellIn Items.[j].Quality
    printfn ""
    app.UpdateQuality()

-------- day 0 --------
name, sellIn, quality
+5 Dexterity Vest, 10, 20
Aged Brie, 2, 0
Elixir of the Mongoose, 5, 7
Sulfuras, Hand of Ragnaros, 0, 80
Sulfuras, Hand of Ragnaros, -1, 80
Backstage passes to a TAFKAL80ETC concert, 15, 20
Backstage passes to a TAFKAL80ETC concert, 10, 49
Backstage passes to a TAFKAL80ETC concert, 5, 49
Conjured Mana Cake, 3, 6

-------- day 1 --------
name, sellIn, quality
+5 Dexterity Vest, 9, 19
Aged Brie, 1, 1
Elixir of the Mongoose, 4, 6
Sulfuras, Hand of Ragnaros, 0, 80
Sulfuras, Hand of Ragnaros, -1, 80
Backstage passes to a TAFKAL80ETC concert, 14, 21
Backstage passes to a TAFKAL80ETC concert, 9, 50
Backstage passes to a TAFKAL80ETC concert, 4, 50
Conjured Mana Cake, 2, 5

-------- day 2 --------
name, sellIn, quality
+5 Dexterity Vest, 8, 18
Aged Brie, 0, 2
Elixir of the Mongoose, 3, 5
Sulfuras, Hand of Ragnaros, 0, 80
Sulfuras, Hand of Ragnaros, -1, 80
Backstage passes to a TAFKAL80ETC concert, 13, 22
Backstage passes to a TAFKAL80ETC concert, 8,